# 杨娟2020

机会和成本对农村学生接受更多教育的影响

## 摘要

近年来，随着代际流动性的固化和影子教育的加剧，教育部出台了一系列旨在提高农村学生受教育程度的政策措施。本文运用２０１５年１％人口抽样调查数据，从教育机会和成本的角度分别评估中职扩招和中职免学费政策对于农村学生上高中的效果。研究发现，两项政策皆对农村学生接受教育产生了影响，并且政策效应随时间的推移而逐步增强。其中，中职扩招和免学费政策都对农村学生读中职和大专产生了促进作用，即政策会对农村学生的受教育状况产生更为有益的深远影响。进一步研究发现，各政策影响不同地区农村学生接受教育的情况具有差异，西部农村学生受到的政策效应最小。

## Points

中职扩招和中职免学费政策 vs 农村学生上高中的效果

技术：扩招政策评估，did

数据：普查2015

## 创新

1. 新数据（2015普查）
2. 方法好一点（did）


In [2]:
import pandas as pd
import numpy as np
import os 

import sys
sys.path.insert(0, '..')  # 将上一层目录添加到系统路径

data_path = os.getenv('data_path')

from src import reader


In [3]:
file_path = os.path.join(data_path,'census\\2015\\Census2015.dta')

df = reader.read_data(file_path)

C:\Users\lee\nutcloud\Data\census\2015\Census2015.dta
feather存在，读取feather。


In [4]:
df.iloc[:5,:5]

,M1,M2,M3,M7,M8
0,章贡区,360702,7.0,1,1
1,九江县,360421,6.0,1,1
2,温江区,510115,2.0,1,1
3,市中区,370103,4.0,1,1
4,永善县,530625,2.0,1,1


## 模型

$$
Y_{it} = \beta_0 + \beta_1 policy_t + \beta_2 group_i + \beta_3 policy_t \times group_i + \sum \gamma x_i + \phi_i + \tau_t + \varepsilon_{i,t}
$$

使用线性模型，便于解释。稳健性检验中使用logit模型。


### 政策时间点

1. 2002年中职扩招（和普通高中平行），影响2002年和之后的初中升高中的个体。
2. 2009年12月中职免费，影响2010年和之后的个体。

### 分组

1. 农村户口作为处理组。
2. 城市户口作为对照组。

### 控制变量

1. 性别、民族、是否为独生子女、父母受教育年限、中职扩招速度
2. 省份固定效应，年份固定效应


## 数据：控制组和处理组

假定所有人都是6岁入学。

问：中职扩展是2002年，影响这一届的初升高的同学（2002年9月入学高中），应该是那一年出生？

学前 + 小学 + 初中 = 6 + 6 + 3 = 15年

因此：在2002年9月1日时，年龄>=15，且<16的人，会在此时入学高一。

最早：2002年09月01日 - 16年 = 1986年9月1日 

最晚：2002年09月01日 - 15年 - 1天 = 1987年8月31日 

1. 考虑在8月31日23点59分59秒出生，在2002年09月01日时候是15周岁01秒
2. 考虑在9月1日0点0分1秒出生，晚了2秒，比15周岁少了1秒，不到15周岁。
3. 因此，最晚不能晚于8月31日。

所以，这次入学高一的同学的出生时间是：1986年9月1日 ~ 1987年8月31日。

因此，政策前样本为 1986年8月31日及之前出生的人，1986年9月1日及之后为政策后出生的人。


In [21]:
def affected_birth_date(enroll_year, enroll_level = '高中'):
    """计算受到入学政策影响人的具体出生年月"""
    years = {'学前':6,
            '小学':6,
                '初中':3,
                '高中':3,
                '大学':4}

    years[enroll_level] # 测试key在不在

    total_years = 0
    for l, y in years:
        if l == enroll_level:
            break
        total_years += years[enroll_level]

    msg = f"""1. 第一批受影响样本：{enroll_year - total_years - 1}年9月1日 到 {enroll_year - total_years}年8月31日
    2. 不受影响样本：{enroll_year - total_years - 1}年8月31日 及之前
    3. 受到影响样本：{enroll_year - total_years - 1}年9月1日 及之后
    """

    # 使用.splitlines()将字符串分割成行的列表，然后使用列表推导去掉每行前面的空白字符
    cleaned_lines = [line.lstrip() for line in msg.splitlines()]
    cleaned_string = '\n'.join(cleaned_lines)

    return cleaned_string

# 
print(affected_birth_date(2002, '高中'))

1. 第一批受影响样本：1986年9月1日 到 1987年8月31日
2. 不受影响样本：1986年8月31日 及之前
3. 受到影响样本：1986年9月1日 及之后

